In [1]:
# imports
import os
import torch
import torch.nn as nn
import json
import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image, read_file

In [2]:
dataset_path = '../dataset_full/' # to modify according to your folder

In [3]:
# check to see if we have good number of pictures
dir_path = r'../dataset_full/img_resized'
# print(os.listdir(dir_path))
count = 0
for item in os.listdir(dir_path):
    # check if current path is a file
    count += 1
print('File count:', count)

File count: 150000


In [4]:
anns = json.load(open(dataset_path + 'MMHS150K_GT.json',"r"))
print("Loaded anns: " + str(len(anns)))

majority_not_hate = 0
majority_hate = 0
majority_racist = 0
majority_sexist = 0
majority_homo = 0
majority_religion = 0
majority_other = 0

for k,v in anns.items():
    labels = []
    label_num = []
    # print(len(v["labels_str"]))
    for label in v["labels_str"]:
        if "Not" in label:
            label_num.append(0)
        elif "Racist" in label:
            label_num.append(1)
        elif "Sexist" in label:
            label_num.append(2)
        elif "Homo" in label:
            label_num.append(3)
        elif "Religion" in label:
            label_num.append(4)
        elif "Other" in label:
            label_num.append(5)
        else:
            print("Error with: " + label)
            label = "Error"

    if label_num.count(0) > 1:
        majority_not_hate+=1
    else:
        majority_hate+=1
        if label_num.count(1) > 1:
            majority_racist+=1
        elif label_num.count(2) > 1:
            majority_sexist+=1
        elif label_num.count(3) > 1:
            majority_homo+=1
        elif label_num.count(4) > 1:
            majority_religion+=1
        elif label_num.count(5) > 1:
            majority_other+=1

print("Total Tweets Majority Voting: Not Hate: " + str(majority_not_hate) + ", Hate: " + str(majority_hate) + ", Racist: " + str(majority_racist) + ", Sexist: " + str(majority_sexist) + ", Homophobe: " + str(majority_homo) + ", Religion: " + str(majority_religion) + ", Other: " + str(majority_other))

Loaded anns: 149823
Total Tweets Majority Voting: Not Hate: 112845, Hate: 36978, Racist: 11925, Sexist: 3495, Homophobe: 3870, Religion: 163, Other: 5811


In [5]:
image_path = os.path.join(dataset_path, 'img_resized')
text_path = os.path.join(dataset_path, 'img_txt')
GT_path = os.path.join(dataset_path, 'MMHS150K_GT.csv')

In [6]:
print(GT_path)

../dataset_full/MMHS150K_GT.csv


In [7]:
from preprocessing import create_csv_labels
# uncomment the first time
# create_csv_labels(os.path.join(dataset_path, 'MMHS150K_GT.json'), GT_path, text_path)

In [61]:

# from PIL import Image
# output_path =  os.path.join(dataset_path, 'img_resized_2')
# print(output_path)
# class MMHS_150KDataset(Dataset):
#     """MMHS_150K dataset."""

#     def __init__(self, GT_path, image_path, Rescale=None, transform=None):
#         """
#         Arguments:
#             GT_path (string): Path to the json file with annotations.
#             root_dir (string): Directory with all the images.
#             transform (callable, optional): Optional transform to be applied
#                 on a sample.
#         """
#         self.GT_path = GT_path
#         self.GT_data = pd.read_csv(GT_path)
#         self.idx_list = []
#         self.output_size = Rescale
#         (self.new_width, self.new_height) = (Rescale, Rescale)
        
#         self.image_path = image_path
#         self.transform = transform

#         self.refine_images()

#     def __len__(self):
#         return self.len_samples

#     def __getitem__(self, idx):
#         if torch.is_tensor(idx):
#             idx = idx.tolist()

#         ID = self.GT_data.iloc[idx, 0]
#         # txt_path = os.path.join(self.text_path, str(ID) + '.json').replace("\\","/")
#         img_path = os.path.join(self.image_path, str(ID) + '.jpg').replace("\\","/")
        
#         image = read_image(img_path)

#         text = self.GT_data.iloc[idx, 3]

#         label = self.GT_data.iloc[idx, 2]

#         sample = {'ID': ID, 'text': text, 'image': image, 'label': label}
#         if self.transform:
#             sample = self.transform(sample)

#         return sample

#     def refine_images(self):
#       GT_path_cleared = os.path.join(dataset_path, 'MMHS150K_GT_cleared.csv')
#       cmpt = 0
#       with open(GT_path, 'r') as readFile:
#         lines = readFile.readlines()
#         for row in lines[1:]:
#           idx = row[0:19]
#           img_path = os.path.join(self.image_path, str(idx) + '.jpg').replace("\\","/")
#           # look if the image exists, otherwise delete the idx from the csv
#           if not (os.path.isfile(img_path)):
#             lines.remove(row)
#             cmpt = cmpt + 1

#       with open(GT_path_cleared, 'w') as writeFile:
#         for line in lines:
#           writeFile.write(line)

#       self.len_samples = len(lines)
#       self.GT_data =  pd.read_csv(GT_path_cleared)
#       # print(f"nbr_missing = {cmpt}")
#       # print(f"len = {len(self.GT_data.iloc[:, 0])}")
#       # print(f"linelen = {len(lines)}")

# example = MMHS_150KDataset(GT_path, image_path, Rescale=299)
# # example[100]

../dataset_full/img_resized_2


In [87]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class MMHS_150KDataset(Dataset):
    """MMHS_150K dataset."""

    def __init__(self, GT_path, image_path, Rescale=None, transform=None, save_path=None, create_resized_folder=False):
        """
        Arguments:
            GT_path (string): Path to the json file with annotations.
            image_path (string): Directory with all the images.
            Rescale (int): Desired size for resizing the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            save_path (string): Directory to save the processed images.
        """
        self.GT_path = GT_path
        self.GT_data = pd.read_csv(GT_path)
        self.image_path = image_path
        self.transform = transform

        # Resize parameters
        self.output_size = Rescale
        self.resize_transform = transforms.Resize((Rescale, Rescale))
        self.create_resized_folder = create_resized_folder
        self.save_path = save_path

    def __len__(self):
        return len(self.GT_data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        ID = self.GT_data.iloc[idx, 0]
        img_path = os.path.join(self.image_path, str(ID) + '_processed.jpg').replace("\\","/")

        ############################################
        if self.save_path:
            self.create_resized_images()
        ############################################

        image = read_image(img_path)

        text = self.GT_data.iloc[idx, 3]
        label = self.GT_data.iloc[idx, 2]

        sample = {'ID': ID, 'text': text, 'image': image, 'label': label}
        if self.transform:
            sample = self.transform(sample)

        return sample
    
    def resize_image(self, image):
        # Resize the image to maintain aspect ratio and fit into (500, 500)
        width, height = image.size
        aspect_ratio = width / height

        resized_image = self.resize_transform(image)
        
        # Crop the image to (500, 500)
        center_crop = transforms.CenterCrop((500, 500))
        cropped_image = center_crop(resized_image)

        return cropped_image
    
    def create_resized_images(self, img_path):
        image = Image.open(img_path).convert('RGB')

        # Resize and crop image
        image = self.resize_image(image)

        # Save processed image
        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)
        save_img_path = os.path.join(self.save_path, str(ID) + '_processed.jpg')
        image.save(save_img_path)


In [89]:
#########################################
## CELL TO RUN TO SAVE IMAGES IN DESIRED FOLDER
#########################################

#specify HERE the path where you wish to save the processed images
image_path =  os.path.join(dataset_path, 'img_resized_2')

dataset = MMHS_150KDataset(GT_path, image_path, Rescale=500, save_path=True)
for idx in range(len(dataset)):
    sample = dataset[idx]

#########################################
#########################################

5321


In [90]:
from torch.utils.data.sampler import SubsetRandomSampler
# dataset = MMHS_150KDataset(GT_path, image_path, Rescale=299)
img_size = 299
batch_size = 16
validation_split = .1
test_split = .5 # corresponds to half ot the validation set
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
print(dataset_size)
indices = list(range(dataset_size))
validation_split = int(np.floor(validation_split * dataset_size))
test_split = int(np.floor(test_split * validation_split))
print(validation_split, test_split)
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices, test_indices = indices[validation_split:], indices[test_split:validation_split], indices[:test_split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                sampler=test_sampler)

59252
5925 2962


In [91]:
next(iter(train_loader))

{'ID': tensor([1114737531403157504, 1113601326901026818, 1058910338870140928,
         1106671930705432577, 1109824530644721664, 1109213343225647106,
         1117055518135267334, 1109104313962557441, 1113604477121105920,
         1061375017919959040, 1063469975942426624, 1113390973302845445,
         1109968509503750145, 1105241583149113345, 1107273212793184256,
         1108022963616575489]),
 'text': ['fuck another nigga ima fuck yo sista <url>',
  'lmfao nigga cristian stupid <url>',
  'look at this retarded shit i found <url>',
  '<user> nigga ain’t even hit puberty yet <url>',
  '<user> nigga! u know i’m always ready!  we ain’t wearin that but shit they got on tho  <repeat> <url>',
  '<user> mileena gang nigga  what’s hood ? <repeat> <url>',
  '<user> nigga  no  <url>',
  "<user> <user> <user> get lost zionazi you're blocked <url>",
  'why was i just in a party with a luxery nigga then this happens <url>',
  'when your baby finally come out the womb but it looks a little retarded

In [92]:
len(train_loader), len(validation_loader), len(test_loader)

(3333, 186, 186)